In [1]:
from math import e, factorial,log, gamma, sqrt, floor
from matplotlib import pyplot as pt
from numpy.random import geometric,exponential,poisson
from scipy.stats import ks_2samp
from scipy.stats import norm
from numpy import linspace
from IPython.display import Markdown
from analysis import *

# Load data

This is a comment


In [2]:
f1 = open("r1-d0-result.txt")
raw1 = f1.read()
f1.close()

f2 = open("r1-d1-result.txt")
raw2 = f2.read()
f2.close()

f3 = open("r5-d0-result.txt")
raw3 = f3.read()
f3.close()

f4 = open("r5-d1-result.txt")
raw4 = f4.read()
f4.close()

f5 = open("r9-d0-result.txt")
raw5 = f5.read()
f5.close()

f6 = open("r9-d1-result.txt")
raw6 = f6.read()
f6.close()

f7 = open("r7-d0-result.txt")
raw7 = f7.read()
f7.close()

f8 = open("r7-d1-result.txt")
raw8 = f8.read()
f8.close()

In [3]:
Ttx = 15
Trx = 60
Tn = 100
    
def compute_histogram(r1,r2):

    Eb,Es,En = computeEvents(r1,Ttx,Trx,Tn)

    display(Markdown('<span style="color: #af0000">'+"P(S) = %.6f"%(Trx*Es/len(r1))+'</span>'))
    display(Markdown('<span style="color: #00af00">'+"P(N) = %.6f"%(Tn*En/len(r1))+'</span>'))
    display(Markdown('<span style="color: #0000af">'+"P(B) = %.6f"%(Ttx*Eb/len(r1))+'</span>'))

    l1 = 1000*Eb / (len(r1))

    print("r(B)(Events/s) = %.6f"%(l1))
    
    hist1 = genWindowHistogram(r1,r2,Trx,Eb)
    print("Success probability evts: %.6f"%(checkSuccess(r2,r1)[0]/Eb))
    print("Success probability= %.6f"%sum(hist1[1:]))
    return hist1,Eb,Es,En

In [4]:
hist1,Eb1,Es1,En1 = compute_histogram(raw1,raw2)
print('---------------------')
hist2,Eb2,Es2,En2 = compute_histogram(raw3,raw4)
print('---------------------')
hist3,Eb3,Es3,En3 = compute_histogram(raw5,raw6)
print('---------------------')
hist4,Eb4,Es4,En4 = compute_histogram(raw7,raw8)
print('---------------------')

geom_d1 = geom_dist(hist1[0],len(hist1))
geom_d2 = geom_dist(hist2[0],len(hist2))
geom_d3 = geom_dist(hist3[0],len(hist3))
geom_d4 = geom_dist(hist4[0],len(hist4))

#pt.bar(range(len(wnds)),wnds)
#pt.axis([0,len(wnds),0,0.15])
#pt.xticks(range(len(wnds)), [str(n) for n in range(len(wnds))])
l = 200
pt.bar(range(0,l),wnds[0:l])
pt.axis([0,l,0,0.06])
pt.xticks(range(0,l), [str(n) for n in range(l)])
pt.show()


<span style="color: #af0000">P(S) = 0.307728</span>

<span style="color: #00af00">P(N) = 0.646121</span>

<span style="color: #0000af">P(B) = 0.692272</span>

r(B)(Events/s) = 46.151479
1.0
Success probability evts: 0.307841
Success probability= 0.394868
---------------------


<span style="color: #af0000">P(S) = 0.800259</span>

...

<span style="color: #af0000">P(S) = 0.903104</span>

<span style="color: #00af00">P(N) = 0.090436</span>

<span style="color: #0000af">P(B) = 0.096896</span>

r(B)(Events/s) = 6.459718
0.9999999999999999
Success probability evts: 0.901222
Success probability= 0.275829
---------------------


NameError: name 'wnds' is not defined

In [ ]:
def check_distribution(hist,rt):
    rvd = poisson_dist(rt,len(hist1))
    print("RMSE Poisson= %.6f"%rmse(rvd,hist)) 
    geom_d = geom_dist(hist[0],len(hist))
    print(hist[0])
    print("RMSE Geom= %.6f"%rmse(geom_d,hist))
    
check_distribution(hist1, rt1)
print('---------------------')
check_distribution(hist2, rt2)
print('---------------------')
check_distribution(hist3, rt3)
print('---------------------')
check_distribution(hist4, rt4)
print('---------------------')
check_distribution(wnds, 91)

In [ ]:
pt.rcParams["figure.figsize"] = (20,10)
pt.plot(range(len(hist1)),hist1,label="P(Rx)= %.3f,T(B)/T(Total)= %.2f%%"%(sum(hist1[1:]),100*Ttx*Eb1/len(raw1)))
pt.plot(range(len(hist2)),hist2,label="P(Rx)= %.3f,T(B)/T(Total)= %.2f%%"%(sum(hist2[1:]),100*Ttx*Eb2/len(raw2)))
pt.plot(range(len(hist3)),hist3,label="P(Rx)= %.3f,T(B)/T(Total)= %.2f%%"%(sum(hist3[1:]),100*Ttx*Eb3/len(raw3)))
pt.plot(range(len(hist4)),hist4,label="P(Rx)= %.3f,T(B)/T(Total)= %.2f%%"%(sum(hist4[1:]),100*Ttx*Eb4/len(raw4)))

pt.legend()
pt.axis([0,len(hist1),0,1])
pt.xticks(range(len(hist1)), [str(n) for n in range(len(hist1))])
pt.xlabel(r'$\mathcal{k}$', fontsize = 18)
pt.ylabel(r'P(k-messages-received)')
pt.grid(True)
pt.show()